In [2]:
import requests
from bs4 import BeautifulSoup  
import pandas as pd
from urllib.request import Request, urlopen
import time
from selenium.common.exceptions import NoSuchElementException

# nextpit
def getReviews(list_of_page):
    reviews = []
    for page_url in list_of_page:
        page = requests.get(page_url)    
        if page.status_code==200:   # Check if success
            soup = BeautifulSoup(page.content, 'html.parser')
            raw_header = soup.find('span', {'class': 'article-title__text'})
            header = raw_header.get_text().strip()
            raw_review = soup.find('div', {'id': 'articleMainContent'}) 
            review = ''
            for p in raw_review.find_all('p'):
                    review = review + p.get_text().strip()

            if 'review' not in header:
                name_review = (page_url.split('/')[-1]).split('-')[:-1]
                name = " ".join(name_review)
            else:
                name = header.split(" review")[0]
            raw_author = soup.find('span', {'class': 'articleAuthorLinkText'})
            author = raw_author.get_text().strip()

            reviews.append({ 'Phone Name': name, 'Healine': header, 'Review': review, 
                            'Source': 'nextpit', 'URL': page_url, 'Author': author})
    df = pd.DataFrame(reviews)
    return df

def getListOfPhone(page_url, base_url):
    links = []
    page = requests.get(page_url) # request current page
    soup = BeautifulSoup(page.content, 'html.parser') # parse HTML content
    links = fetchLinks(soup, links, base_url)
    
    next_button = soup.find('a', {'class': 'arrow'})
    next_link = base_url+next_button.get('href')
    page = requests.get(next_link) # request current page
    soup = BeautifulSoup(page.content, 'html.parser') # parse HTML content
    links = fetchLinks(soup, links, base_url)
    
    count = 0
    while True:
        try:
            time.sleep(0.05)
            next_button = soup.find_all('a', {'class': 'arrow'})
            if(len(next_button) > 1):
                next_button = next_button[1]
            else:
                break
            next_link = base_url+next_button.get('href')
        except NoSuchElementException:
            break # stop after last page
        page = requests.get(next_link) # request current page
        soup = BeautifulSoup(page.content, 'html.parser') # parse HTML content
        links = fetchLinks(soup, links, base_url)
        count = count + 1

    return links  

def fetchLinks(soup, links, base_url):
    anchors = soup.find_all('a', {'class': 'articleTeaser__img-link'})
    for link in anchors:
        if 'review' in link.get('href'):
            links.append(base_url+link.get('href'))
    return links

page_url = 'https://www.nextpit.com/smartphone/reviews'
base_url = 'https://www.nextpit.com'
list_of_page = getListOfPhone(page_url, base_url)

df = getReviews(list_of_page)
df.to_csv('CSVs/nextpit.csv')
df

,Phone Name,Healine,Review,Source,URL,Author
0,oppo a16s,This Oppo smartphone is cheaper than your AirPods,The Oppo A16s costs less than €140 - around $1...,nextpit,https://www.nextpit.com/oppo-a16s-review,Stefan Möllenhoff
1,Google Pixel 6 Pro,Google Pixel 6 Pro review: Excellent and (mayb...,Google has a flagship and it goes by the name ...,nextpit,https://www.nextpit.com/google-pixel-6-pro-review,Camila Rinaldi
2,Google Pixel 6,Google Pixel 6 review: A new hope,The Pixel 6 is the smartphone that is supposed...,nextpit,https://www.nextpit.com/google-pixel-6-review,Antoine Engels
3,ColorOS 12,ColorOS 12 review: My take on Oppo's Android 1...,ColorOS 12 is the new interface for Oppo smart...,nextpit,https://www.nextpit.com/oppo-coloros-12-review,Antoine Engels
4,iPhone 13 Pro,iPhone 13 Pro review: What unlucky number?,"At first glance, the iPhone 13 Pro looks like ...",nextpit,https://www.nextpit.com/apple-iphone-13-pro-re...,Fabien Roehlinger
...,...,...,...,...,...,...
550,Xperia Z Ultra hands-on,Xperia Z Ultra hands-on review,Phablets are getting to be all the rage. They’...,nextpit,https://www.nextpit.com/xperia-z-ultra-hands-o...,Loie Favre
551,miui v5 custom rom,MIUI v5 Review: a Crowd-Pleasing Custom ROM,Custom ROMS are great: with very little effort...,nextpit,https://www.nextpit.com/miui-v5-custom-rom-review,Loie Favre
552,nexus 4 review how does it compare to the galaxy,Nexus 4 Review: How Does It Compare to the Gal...,"After a long, long wait, a Nexus 4 finally arr...",nextpit,https://www.nextpit.com/nexus-4-review-how-doe...,Deactivated Account
553,androidpit nexus 7,AndroidPIT’s Hands On Review Of The Google Nex...,I’m personally not a huge fan of 7 inch tablet...,nextpit,https://www.nextpit.com/androidpit-nexus-7-review,Eric McBride
